# NBA Game Prediction - Feature Engineering

**Purpose:** Extract and engineer features from RDS database for ML model training

**Data Source:** PostgreSQL RDS (nba_simulator database)

**Output:** Parquet files in S3 with engineered features

---

## Overview

This notebook performs:
1. **Data Validation:** Check RDS data quality
2. **Team Features:** Win rate, PPG, FG%, defensive stats
3. **Player Features:** Usage rate, efficiency, PER
4. **Time Features:** Momentum, streaks, rest days
5. **Categorical Encoding:** Home/away, opponent, venue
6. **S3 Storage:** Save features as Parquet

---

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import boto3
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Data Validation (Workflow #21)

Validate RDS data quality before feature engineering.

In [ ]:
# Database connection
DB_HOST = 'nba-sim-db.ck96ciigs7fy.us-east-1.rds.amazonaws.com'
DB_NAME = 'nba_simulator'
DB_USER = 'postgres'
DB_PASSWORD = 'YOUR_PASSWORD_HERE'  # Replace with actual password

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/{DB_NAME}')

print("✓ Database connection created")

In [ ]:
# Validate data counts
validation_queries = {
    'games': 'SELECT COUNT(*) as count FROM games',
    'play_by_play': 'SELECT COUNT(*) as count FROM play_by_play',
    'player_game_stats': 'SELECT COUNT(*) as count FROM player_game_stats',
    'team_game_stats': 'SELECT COUNT(*) as count FROM team_game_stats'
}

print("Data Validation Results:")
print("=" * 50)
for table, query in validation_queries.items():
    result = pd.read_sql(query, engine)
    print(f"{table:20s}: {result['count'][0]:>10,} rows")

# Expected:
# games: 46,595
# play_by_play: 6,781,155
# player_game_stats: 408,833
# team_game_stats: 15,900

In [ ]:
# Check for missing values in critical fields
games_sample = pd.read_sql("""
    SELECT game_id, game_date, home_team_id, away_team_id, 
           home_points, away_points, season
    FROM games
    LIMIT 10000
""", engine)

print("\nMissing Value Check (sample of 10,000 games):")
print("=" * 50)
missing = games_sample.isnull().sum()
print(missing[missing > 0])
if missing.sum() == 0:
    print("✓ No missing values in critical fields")

In [ ]:
# Check date range
date_range = pd.read_sql("""
    SELECT MIN(game_date) as earliest, MAX(game_date) as latest,
           COUNT(DISTINCT season) as num_seasons
    FROM games
""", engine)

print("\nDate Range:")
print("=" * 50)
print(f"Earliest game: {date_range['earliest'][0]}")
print(f"Latest game:   {date_range['latest'][0]}")
print(f"Total seasons: {date_range['num_seasons'][0]}")
print("✓ Data validation complete")

## 2. Load Game Data

Load complete games table for feature engineering.

In [ ]:
# Load all games with relevant fields
games_query = """
    SELECT 
        game_id, game_date, season, season_type,
        home_team_id, away_team_id,
        home_points, away_points,
        attendance, venue_name, venue_city,
        CASE WHEN home_points > away_points THEN 1 ELSE 0 END as home_win
    FROM games
    WHERE season_type = 2  -- Regular season only
    AND home_points IS NOT NULL
    AND away_points IS NOT NULL
    ORDER BY game_date
"""

print("Loading games data...")
games = pd.read_sql(games_query, engine)
games['game_date'] = pd.to_datetime(games['game_date'])

print(f"✓ Loaded {len(games):,} games")
print(f"  Date range: {games['game_date'].min()} to {games['game_date'].max()}")
print(f"  Seasons: {games['season'].nunique()}")
print(f"  Home win rate: {games['home_win'].mean():.3f}")

## 3. Team Performance Features

Calculate rolling team statistics:
- Win percentage
- Points per game (PPG)
- Points allowed per game
- Recent form (last 10 games)

In [ ]:
def calculate_rolling_stats(games_df, team_id, is_home=True, window=10):
    """
    Calculate rolling statistics for a team.
    
    Args:
        games_df: DataFrame of games
        team_id: Team ID to calculate stats for
        is_home: Whether to calculate home or away stats
        window: Rolling window size (default 10 games)
    
    Returns:
        DataFrame with rolling statistics
    """
    if is_home:
        team_games = games_df[games_df['home_team_id'] == team_id].copy()
        team_games['points_for'] = team_games['home_points']
        team_games['points_against'] = team_games['away_points']
        team_games['won'] = team_games['home_win']
    else:
        team_games = games_df[games_df['away_team_id'] == team_id].copy()
        team_games['points_for'] = team_games['away_points']
        team_games['points_against'] = team_games['home_points']
        team_games['won'] = 1 - team_games['home_win']
    
    team_games = team_games.sort_values('game_date')
    
    # Calculate rolling statistics (using past games only)
    team_games['rolling_win_pct'] = team_games['won'].rolling(window=window, min_periods=1).mean().shift(1)
    team_games['rolling_ppg'] = team_games['points_for'].rolling(window=window, min_periods=1).mean().shift(1)
    team_games['rolling_papg'] = team_games['points_against'].rolling(window=window, min_periods=1).mean().shift(1)
    team_games['rolling_margin'] = team_games['rolling_ppg'] - team_games['rolling_papg']
    
    return team_games[['game_id', 'rolling_win_pct', 'rolling_ppg', 'rolling_papg', 'rolling_margin']]

print("✓ Rolling stats function defined")

In [ ]:
# Calculate rolling stats for all teams
print("Calculating team rolling statistics...")
print("This may take a few minutes...\n")

all_teams = pd.concat([
    games['home_team_id'],
    games['away_team_id']
]).unique()

home_stats_list = []
away_stats_list = []

for i, team_id in enumerate(all_teams, 1):
    if i % 10 == 0:
        print(f"  Processed {i}/{len(all_teams)} teams...")
    
    # Home stats
    home_stats = calculate_rolling_stats(games, team_id, is_home=True)
    home_stats = home_stats.rename(columns={
        'rolling_win_pct': 'home_rolling_win_pct',
        'rolling_ppg': 'home_rolling_ppg',
        'rolling_papg': 'home_rolling_papg',
        'rolling_margin': 'home_rolling_margin'
    })
    home_stats_list.append(home_stats)
    
    # Away stats
    away_stats = calculate_rolling_stats(games, team_id, is_home=False)
    away_stats = away_stats.rename(columns={
        'rolling_win_pct': 'away_rolling_win_pct',
        'rolling_ppg': 'away_rolling_ppg',
        'rolling_papg': 'away_rolling_papg',
        'rolling_margin': 'away_rolling_margin'
    })
    away_stats_list.append(away_stats)

# Combine all stats
home_stats_df = pd.concat(home_stats_list, ignore_index=True)
away_stats_df = pd.concat(away_stats_list, ignore_index=True)

print(f"\n✓ Calculated rolling stats for {len(all_teams)} teams")

In [ ]:
# Merge rolling stats back to games
games_with_stats = games.merge(home_stats_df, on='game_id', how='left')
games_with_stats = games_with_stats.merge(away_stats_df, on='game_id', how='left')

print(f"✓ Merged rolling stats into games DataFrame")
print(f"  Shape: {games_with_stats.shape}")
print(f"  Columns: {games_with_stats.columns.tolist()}")

## 4. Time-Series Features

Calculate:
- Rest days since last game
- Back-to-back game indicator
- Win/loss streaks

In [ ]:
def calculate_rest_days(games_df, team_id, is_home=True):
    """
    Calculate rest days since last game for a team.
    """
    if is_home:
        team_games = games_df[games_df['home_team_id'] == team_id].copy()
    else:
        team_games = games_df[games_df['away_team_id'] == team_id].copy()
    
    team_games = team_games.sort_values('game_date')
    team_games['rest_days'] = team_games['game_date'].diff().dt.days - 1
    team_games['rest_days'] = team_games['rest_days'].fillna(7)  # First game default
    team_games['back_to_back'] = (team_games['rest_days'] == 0).astype(int)
    
    return team_games[['game_id', 'rest_days', 'back_to_back']]

print("Calculating rest days and back-to-backs...")

home_rest_list = []
away_rest_list = []

for team_id in all_teams:
    # Home rest
    home_rest = calculate_rest_days(games, team_id, is_home=True)
    home_rest = home_rest.rename(columns={
        'rest_days': 'home_rest_days',
        'back_to_back': 'home_back_to_back'
    })
    home_rest_list.append(home_rest)
    
    # Away rest
    away_rest = calculate_rest_days(games, team_id, is_home=False)
    away_rest = away_rest.rename(columns={
        'rest_days': 'away_rest_days',
        'back_to_back': 'away_back_to_back'
    })
    away_rest_list.append(away_rest)

home_rest_df = pd.concat(home_rest_list, ignore_index=True)
away_rest_df = pd.concat(away_rest_list, ignore_index=True)

games_with_stats = games_with_stats.merge(home_rest_df, on='game_id', how='left')
games_with_stats = games_with_stats.merge(away_rest_df, on='game_id', how='left')

print(f"✓ Added rest days and back-to-back features")

## 5. Categorical Encoding

Encode categorical variables:
- Season (ordinal)
- Month of season
- Day of week

In [ ]:
# Date-based features
games_with_stats['month'] = games_with_stats['game_date'].dt.month
games_with_stats['day_of_week'] = games_with_stats['game_date'].dt.dayofweek
games_with_stats['is_weekend'] = (games_with_stats['day_of_week'] >= 5).astype(int)

# Season phase (early/mid/late season)
games_with_stats['season_month'] = games_with_stats['game_date'].dt.month
games_with_stats['season_phase'] = pd.cut(
    games_with_stats['season_month'],
    bins=[0, 12, 2, 5],  # Oct-Dec, Jan-Feb, Mar-Apr
    labels=[0, 1, 2],     # Early, mid, late
    ordered=True
)
games_with_stats['season_phase'] = games_with_stats['season_phase'].astype(int)

print("✓ Added categorical features")
print(f"  Months: {games_with_stats['month'].nunique()}")
print(f"  Days of week: {games_with_stats['day_of_week'].nunique()}")
print(f"  Season phases: {games_with_stats['season_phase'].nunique()}")

## 6. Feature Selection & Cleanup

Select final features for model training.

In [ ]:
# Select features for ML model
feature_columns = [
    # Identifiers
    'game_id', 'game_date', 'season',
    'home_team_id', 'away_team_id',
    
    # Target variable
    'home_win',
    
    # Home team features
    'home_rolling_win_pct', 'home_rolling_ppg', 
    'home_rolling_papg', 'home_rolling_margin',
    'home_rest_days', 'home_back_to_back',
    
    # Away team features
    'away_rolling_win_pct', 'away_rolling_ppg',
    'away_rolling_papg', 'away_rolling_margin',
    'away_rest_days', 'away_back_to_back',
    
    # Categorical features
    'month', 'day_of_week', 'is_weekend', 'season_phase'
]

features_df = games_with_stats[feature_columns].copy()

print(f"✓ Selected {len(feature_columns)} feature columns")
print(f"  Total rows: {len(features_df):,}")

In [ ]:
# Remove rows with missing values (first few games per team)
features_df_clean = features_df.dropna()

print(f"✓ Removed rows with missing values")
print(f"  Before: {len(features_df):,} rows")
print(f"  After:  {len(features_df_clean):,} rows")
print(f"  Dropped: {len(features_df) - len(features_df_clean):,} rows ({(1 - len(features_df_clean)/len(features_df))*100:.1f}%)")

In [ ]:
# Data quality checks
print("\nFinal Feature Summary:")
print("=" * 70)
print(features_df_clean.describe())

print("\nFeature Correlations with Target (home_win):")
print("=" * 70)
numeric_cols = features_df_clean.select_dtypes(include=[np.number]).columns
correlations = features_df_clean[numeric_cols].corr()['home_win'].sort_values(ascending=False)
print(correlations)

## 7. Save to S3 (Parquet Format)

Save engineered features to S3 for model training.

In [ ]:
# S3 configuration
S3_BUCKET = 'nba-sim-raw-data-lake'
S3_PREFIX = 'ml-features'

# Save to S3 as Parquet
output_path = f's3://{S3_BUCKET}/{S3_PREFIX}/game_features.parquet'

print(f"Saving features to S3...")
print(f"  Path: {output_path}")
print(f"  Rows: {len(features_df_clean):,}")
print(f"  Columns: {len(features_df_clean.columns)}")

features_df_clean.to_parquet(output_path, engine='pyarrow', index=False)

print("✓ Features saved to S3")

In [ ]:
# Verify S3 upload
s3 = boto3.client('s3')

try:
    response = s3.head_object(Bucket=S3_BUCKET, Key=f'{S3_PREFIX}/game_features.parquet')
    file_size_mb = response['ContentLength'] / (1024 * 1024)
    
    print("\nS3 Upload Verification:")
    print("=" * 70)
    print(f"✓ File exists in S3")
    print(f"  Size: {file_size_mb:.2f} MB")
    print(f"  Last modified: {response['LastModified']}")
    print(f"  Full path: s3://{S3_BUCKET}/{S3_PREFIX}/game_features.parquet")
except Exception as e:
    print(f"✗ Error verifying S3 upload: {e}")

## 8. Create Train/Test Split

Split data chronologically (train on earlier seasons, test on recent seasons).

In [ ]:
# Time-based split (80/20)
features_df_clean = features_df_clean.sort_values('game_date')
split_idx = int(len(features_df_clean) * 0.8)

train_df = features_df_clean.iloc[:split_idx]
test_df = features_df_clean.iloc[split_idx:]

print(f"Train/Test Split:")
print("=" * 70)
print(f"Train set:")
print(f"  Rows: {len(train_df):,}")
print(f"  Date range: {train_df['game_date'].min()} to {train_df['game_date'].max()}")
print(f"  Home win rate: {train_df['home_win'].mean():.3f}")
print(f"\nTest set:")
print(f"  Rows: {len(test_df):,}")
print(f"  Date range: {test_df['game_date'].min()} to {test_df['game_date'].max()}")
print(f"  Home win rate: {test_df['home_win'].mean():.3f}")

In [ ]:
# Save train/test splits to S3
train_path = f's3://{S3_BUCKET}/{S3_PREFIX}/train.parquet'
test_path = f's3://{S3_BUCKET}/{S3_PREFIX}/test.parquet'

print("Saving train/test splits to S3...")
train_df.to_parquet(train_path, engine='pyarrow', index=False)
test_df.to_parquet(test_path, engine='pyarrow', index=False)

print(f"✓ Train data: {train_path}")
print(f"✓ Test data:  {test_path}")

## 9. Feature Engineering Summary

Summary of created features and next steps.

In [ ]:
print("\n" + "=" * 70)
print("FEATURE ENGINEERING COMPLETE")
print("=" * 70)
print(f"\nTotal games processed: {len(features_df_clean):,}")
print(f"Total features: {len(feature_columns)}")
print(f"\nFeature categories:")
print(f"  - Team performance (rolling stats): 8 features")
print(f"  - Rest/schedule: 4 features")
print(f"  - Temporal: 4 features")
print(f"  - Target variable: 1 feature (home_win)")
print(f"\nS3 outputs:")
print(f"  - Full dataset: s3://{S3_BUCKET}/{S3_PREFIX}/game_features.parquet")
print(f"  - Train set:    s3://{S3_BUCKET}/{S3_PREFIX}/train.parquet")
print(f"  - Test set:     s3://{S3_BUCKET}/{S3_PREFIX}/test.parquet")
print(f"\nNext steps:")
print(f"  1. Create 03_model_training.ipynb")
print(f"  2. Train baseline models (logistic regression, random forest)")
print(f"  3. Evaluate model performance")
print(f"  4. Tune hyperparameters")
print("=" * 70)